In [ ]:
import time, cv2, csv, datetime
import numpy as np
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from keras.models import load_model
from crawler import refine_coordinate, crop_image
from preprocessBatch import preprocessing

FULL_IMG = "thsrc.png"
WIDTH = 140
HEIGHT = 48
FOLDER = "captcha/"
CSV_FILE = "auto-train.csv"
allowedChars = '234579ACFHKMNPQRTYZ';

print('model loading...')
model = load_model("thsrc_cnn_model.hdf5")
print('loading completed')

In [ ]:
i = 10000
failed = 50

while i < 10001:
    chrome_options = Options() # 啟動無頭模式
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    executable_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)
    driver.get('http://irs.thsrc.com.tw/IMINT/')

    driver.find_element_by_id('btn-confirm').click()
    
    driver.save_screenshot(FULL_IMG)

    element = driver.find_element_by_xpath('//*[@id="BookingS1Form_homeCaptcha_passCode"]')
    location = element.location
    size = element.size
    ratio = int(driver.execute_script('return window.devicePixelRatio'))

    coordinate = refine_coordinate(location, size, ratio)
    img = crop_image(FULL_IMG, coordinate)
    img = img.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
    # convert rgba to rgb
    img = img.convert('RGB')
    img.save('captcha.jpg', "JPEG")

    preprocessing('captcha.jpg', 'preprocessing.jpg')

    train_data = np.stack([np.array(cv2.imread("preprocessing.jpg"))/255.0])
    prediction = model.predict(train_data)

    predict_captcha = ''
    for predict in prediction:
        value = np.argmax(predict[0])
        predict_captcha += allowedChars[value]

#     print(predict_captcha)

    # 填入表單
    startStation = Select(driver.find_element_by_name('selectStartStation'))
    startStation.select_by_value("1") # 南港

    selectDestinationStation = Select(driver.find_element_by_name('selectDestinationStation'))
    selectDestinationStation.select_by_value("2") # 台北
    
    date_next_week = datetime.datetime.today() + datetime.timedelta(days=7)
    toTimeInputField = driver.find_element_by_name("toTimeInputField")
    length = len(toTimeInputField.get_attribute('value'))
    toTimeInputField.send_keys(length * Keys.BACKSPACE)
    toTimeInputField.send_keys(date_next_week.strftime("%Y/%m/%d"))

    toTimeTable = Select(driver.find_element_by_name('toTimeTable'))
    toTimeTable.select_by_index(9) # 09:00

    captchaInput = driver.find_element_by_name("homeCaptcha:securityCode")
    captchaInput.send_keys(predict_captcha)

    driver.find_element_by_id('SubmitButton').click()
    time.sleep(1)
    if driver.title == "台灣高鐵網路訂位 > 選擇車次":
        # add to label
        i += 1
        print("success, add " + str(i) + ".jpg with captcha: " + predict_captcha)
        img.save(FOLDER + str(i) + ".jpg", "JPEG")
        with open(CSV_FILE,'a') as fd:
            fd.write(predict_captcha + "\n")
    else:
        print("predict fail, " + driver.title)
        img.save("failed/" + str(failed) + ".jpg", "JPEG")
        failed += 1

    driver.quit()

print("completed")